# CE 5506 — Introducción al Reconocimiento de Patrones
## Tarea 3
## Redes Neuronales
### Shakime Richards Sparks     - 2018170667
### Carlos Adrián Araya Ramírez - 2018319701

In [1]:
import numpy as np
import pandas as pd
import time

In [2]:
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score

def calculate_recall(predictions, y_real, average='macro'):
    recall = recall_score(y_real, predictions, average=average)
    return recall

def calculate_accuracy(predictions, y_real):
    correct = np.sum(np.array(predictions) == np.array(y_real))
    total = len(y_real)
    accuracy = correct / total
    return accuracy

def calculate_precision(predictions, y_real, average='macro'):
    precision = precision_score(y_real, predictions, average=average, zero_division=1)
    return precision

def calculate_f1_score(predictions, y_real, average='macro'):
    f1 = f1_score(y_real, predictions, average=average)
    return f1

# Parte 1

In [3]:
class PerceptronMulticapa:
    def __init__(self, capas, alpha=0.1):
        self.capas = capas
        self.alpha = alpha
        self.bias = []
        self.pesos = []
        for i in range(0, len(capas) - 1):
            # Inicializar los pesos y bias de cada capa
            peso = np.random.randn(capas[i], capas[i+1])
            self.pesos.append(peso)
            bias = np.random.randn(capas[i+1])
            self.bias.append(bias)

    def activacion(self, x):
        # Función de activación sigmoide
        return 1.0 / (1 + np.exp(-x))

    def activacion_derivada(self, x):
        # Derivada de la función de activación sigmoide
        return x * (1 - x)

    def feedforward(self, X):
        # Calcular la salida de cada capa
        capa_activacion = [X]
        for i in range(0, len(self.capas) - 1):
            x = np.dot(capa_activacion[i], self.pesos[i]) + self.bias[i]
            y = self.activacion(x)
            capa_activacion.append(y)
        return capa_activacion

    def backpropagation(self, X, y, capa_activacion):
        # Calcular el error de la capa de salida
        error = capa_activacion[-1] - y
        delta = error * self.activacion_derivada(capa_activacion[-1])
        
        # Propagar el error hacia atrás a través de la red neuronal
        for i in reversed(range(0, len(self.capas) - 1)):
            activacion_actual = capa_activacion[i]
            d_peso = np.outer(activacion_actual, delta)
            d_bias = delta
            self.pesos[i] -= self.alpha * d_peso
            self.bias[i] -= self.alpha * d_bias
            if (i!=0):
                delta = np.dot(delta, self.pesos[i].T) * self.activacion_derivada(activacion_actual)

    def entrenar(self, X, y, epochs):
        for epoch in range(0, epochs):
            for i in range(0, len(X)):
                # Feedforward
                capa_activacion = self.feedforward(X[i])

                # Backpropagation
                self.backpropagation(X[i], y[i], capa_activacion)

    def predecir(self, X):
        # Obtener la salida de la última capa
        capa_activacion = self.feedforward(X)
        return capa_activacion[-1]
    

In [4]:
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt


# Cargar el conjunto de datos MNIST
mnist = fetch_openml('mnist_784', parser='auto')
X = mnist.data
X = X / np.max(X)
y = mnist.target

# Dividir el conjunto de datos en entrenamiento y prueba
X_entrenamiento, X_prueba, y_entrenamiento, y_prueba = train_test_split(X, y, test_size=0.2, random_state=42)

y_entrenamiento = y_entrenamiento.values.astype(int)
y_prueba = y_prueba.values.astype(int)

# Verificar el tamaño de las matrices
print("Tamaño de X_entrenamiento:", X_entrenamiento.shape)
print("Tamaño de X_prueba:", X_prueba.shape)
print("Tamaño de y_entrenamiento:", y_entrenamiento.shape)
print("Tamaño de y_prueba:", y_prueba.shape)


Tamaño de X_entrenamiento: (56000, 784)
Tamaño de X_prueba: (14000, 784)
Tamaño de y_entrenamiento: (56000,)
Tamaño de y_prueba: (14000,)


# Feature engineering

### Se aplican dos técnicas de feature engineering al conjunto de datos MNIST. Primero, se realiza el escalamiento de características utilizando StandardScaler de scikit-learn para estandarizar las características y asegurarse de que todas tengan la misma escala. Luego, se aplica la reducción de dimensionalidad utilizando PCA (Análisis de Componentes Principales) con el objetivo de conservar el 95% de la varianza de los datos originales.


In [5]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

mu = np.mean(X, 0)
sigma = np.std(X, 0)
X = (X - mu ) / sigma

# Escalamiento de características
scaler = StandardScaler()
X_entrenamiento = scaler.fit_transform(X_entrenamiento)
X_prueba = scaler.transform(X_prueba)

# Reducción de dimensionalidad utilizando PCA
pca = PCA(n_components=0.95)  # Mantener el 95% de la varianza
X_entrenamiento = pca.fit_transform(X_entrenamiento)
X_prueba = pca.transform(X_prueba)

# Verificar el tamaño de las matrices después de aplicar feature engineering
print("Tamaño de X_entrenamiento después de feature engineering:", X_entrenamiento.shape)
print("Tamaño de X_prueba después de feature engineering:", X_prueba.shape)

Tamaño de X_entrenamiento después de feature engineering: (56000, 330)
Tamaño de X_prueba después de feature engineering: (14000, 330)


In [32]:
# Crear y entrenar el perceptrón multicapa
perceptron = PerceptronMulticapa(capas=[330,4,4,10], alpha=0.225)

start_time = time.time()
perceptron.entrenar(X_entrenamiento, np.eye(10)[y_entrenamiento], epochs=1000)
end_time = time.time()

# Hacer predicciones sobre el conjunto de prueba
y_pred = []
for i in range(len(X_prueba)):
    prediccion = perceptron.predecir(X_prueba[i])
    prediccion_clase = np.argmax(prediccion)
    y_pred.append(prediccion_clase)

# Calcular la precisión de las predicciones
print("Predicciones: ", y_pred[0:10])
print("Y_Real      : ", y_prueba[0:10])
precision = sum(y_pred == y_prueba) / len(y_prueba)
print(f"Precisión: {precision}")

/var/folders/8c/zr96_fcx2pqfnygfnn_ysgfm0000gn/T/ipykernel_3037/3044346685.py:16: RuntimeWarning: overflow encountered in exp
  return 1.0 / (1 + np.exp(-x))


Predicciones:  [8, 4, 5, 7, 7, 2, 6, 2, 7, 4]
Y_Real      :  [8 4 8 7 7 0 6 2 7 4]
Precisión: 0.8392857142857143


In [33]:
recall = calculate_recall(y_pred, y_prueba, average='macro')
accuracy = calculate_accuracy(y_pred, y_prueba)
precision = calculate_precision(y_pred, y_prueba, average='macro')
f1 = calculate_f1_score(y_pred, y_prueba, average='macro')
training_time = end_time - start_time

pd.DataFrame({
    'Metric': ['Recall', 'Accuracy', 'Precision', 'F1-score', 'Training time'],
    'Score': [recall, accuracy, precision, f1, training_time]
})

,Metric,Score
0,Recall,0.837372
1,Accuracy,0.839286
2,Precision,0.840565
3,F1-score,0.838077
4,Training time,1757.816665


# Parte 2

In [20]:
class PerceptronMulticapaMultiSize:
    def __init__(self, capas, funciones_capas='sigmoid', alpha=0.1, epochs=1000):
        self.capas = capas
        self.alpha = alpha
        self.bias = []
        self.pesos = []
        self.funciones_capas = funciones_capas
        self.epochs = epochs
        self.derivadas_activacion = {
            'sigmoid': self.activacion_derivada,
            'tanh': self.tanh_activacion_derivada,
            'relu': self.relu_activacion_derivada
        }
        self.activaciones = {
            'sigmoid': self.activacion,
            'tanh': self.tanh_activacion,
            'relu': self.relu_activacion
        }
        for i in range(0, len(capas) - 1):
            # Inicializar los pesos y bias de cada capa
            peso = np.random.randn(capas[i], capas[i+1])
            self.pesos.append(peso)
            bias = np.random.randn(capas[i+1])
            self.bias.append(bias)

    def activacion(self, x):
        # Función de activación sigmoide
        return 1.0 / (1 + np.exp(-x))

    def activacion_derivada(self, x):
        # Derivada de la función de activación sigmoide
        return x * (1 - x)

    def tanh_activacion(self, x):
        # Función de activación tangente hiperbólica (tanh)
        return np.tanh(x)
    
    def tanh_activacion_derivada(self, x):
        # Derivada de la función de activación tangente hiperbólica (tanh)
        return 1 - x **2

    def relu_activacion(self, x):
        # Función de activación ReLU
        return np.maximum(0.01 * x, x)
    
    def relu_activacion_derivada(self, x):
        # Derivada de la función de activación ReLU
        return np.where(x > 0, 1, 0.01)

    def feedforward(self, X):
        # Calcular la salida de cada capa
        capa_activacion = [X]
        for i in range(0, len(self.capas) - 1):
            x = np.dot(capa_activacion[i], self.pesos[i]) + self.bias[i]
            y = self.activaciones[self.funciones_capas[i]](x)
            capa_activacion.append(y)

        return capa_activacion

    def backpropagation(self, X, y, capa_activacion):
       # Calcular el error de la capa de salida
        error = capa_activacion[-1] - y
        delta = error * self.activacion_derivada(capa_activacion[-1])
        
        # Propagar el error hacia atrás a través de la red neuronal
        for i in reversed(range(0, len(self.capas) - 1)):
            activacion_actual = capa_activacion[i]
            d_peso = np.outer(activacion_actual, delta)
            d_bias = delta
            self.pesos[i] -= self.alpha * d_peso
            self.bias[i] -= self.alpha * d_bias
            if (i!=0):
                delta = np.dot(delta, self.pesos[i].T) * self.derivadas_activacion[self.funciones_capas[i]](activacion_actual) 


    def entrenar(self, X, y):
        for epoch in range(0, self.epochs):
            for i in range(0, len(X)):
                # Feedforward
                capa_activacion = self.feedforward(X[i])

                # Backpropagation
                self.backpropagation(X[i], y[i], capa_activacion)

    def predecir(self, X):
        # Obtener la salida de la última capa
        capa_activacion = self.feedforward(X)
        return capa_activacion[-1]
    

In [38]:
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt


# Cargar el conjunto de datos MNIST
mnist = fetch_openml('mnist_784', parser='auto')
X = mnist.data
X = X / np.max(X)
y = mnist.target

# Dividir el conjunto de datos en entrenamiento y prueba
X_entrenamiento, X_prueba, y_entrenamiento, y_prueba = train_test_split(X, y, test_size=0.2, random_state=42)

y_entrenamiento = y_entrenamiento.values.astype(int)
y_prueba = y_prueba.values.astype(int)

# Verificar el tamaño de las matrices
print("Tamaño de X_entrenamiento:", X_entrenamiento.shape)
print("Tamaño de X_prueba:", X_prueba.shape)
print("Tamaño de y_entrenamiento:", y_entrenamiento.shape)
print("Tamaño de y_prueba:", y_prueba.shape)

Tamaño de X_entrenamiento: (56000, 784)
Tamaño de X_prueba: (14000, 784)
Tamaño de y_entrenamiento: (56000,)
Tamaño de y_prueba: (14000,)


In [39]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

mu = np.mean(X, 0)
sigma = np.std(X, 0)
X = (X - mu ) / sigma

# Escalamiento de características
scaler = StandardScaler()
X_entrenamiento = scaler.fit_transform(X_entrenamiento)
X_prueba = scaler.transform(X_prueba)

# Reducción de dimensionalidad utilizando PCA
pca = PCA(n_components=0.95)  # Mantener el 95% de la varianza
X_entrenamiento = pca.fit_transform(X_entrenamiento)
X_prueba = pca.transform(X_prueba)

# Verificar el tamaño de las matrices después de aplicar feature engineering
print("Tamaño de X_entrenamiento después de feature engineering:", X_entrenamiento.shape)
print("Tamaño de X_prueba después de feature engineering:", X_prueba.shape)

Tamaño de X_entrenamiento después de feature engineering: (56000, 330)
Tamaño de X_prueba después de feature engineering: (14000, 330)


## En esta parte se llama la función que se solicita con los siguientes elementos:

- Una lista con la cantidad de neuronas en cada capa oculta
- Una lista con las funciones de activación en cada capa
- Un valor de α y la cantidad de epochs con la que se desea entrenar

Nota 1: Para más simplicidad, no se creó especificamente una función, sino que se asume que el constructor (__init__) de la clase PerceptoMulticapaMultiSize es la función y la misma cuenta con los parámetros requeridos.

Nota 2: En 2 celdas arriba se especifíca la cantidad de datos de entrenamiento (80%): 

X_entrenamiento, X_prueba, y_entrenamiento, y_prueba = train_test_split(X, y, test_size=0.2, random_state=42)

In [42]:
# Crear y entrenar el perceptrón multicapa
perceptronMultiSize = PerceptronMulticapaMultiSize(capas=[330,15,12,32,10], funciones_capas=['tanh', 'relu', 'sigmoid', 'sigmoid'], alpha=0.01233333865, epochs=500)

start_time = time.time()
perceptronMultiSize.entrenar(X_entrenamiento, np.eye(10)[y_entrenamiento])
end_time = time.time()

# Hacer predicciones sobre el conjunto de prueba
y_pred = []
for i in range(len(X_prueba)):
    prediccion = perceptronMultiSize.predecir(X_prueba[i])
    prediccion_clase = np.argmax(prediccion)
    y_pred.append(prediccion_clase)

# Calcular la precisión de las predicciones
print("Predicciones: ", y_pred[0:10])
print("Y_Real      : ", y_prueba[0:10])
precision = sum(y_pred == y_prueba) / len(y_prueba)
print(f"Precisión: {precision}")

/var/folders/8c/zr96_fcx2pqfnygfnn_ysgfm0000gn/T/ipykernel_8315/3970662825.py:28: RuntimeWarning: overflow encountered in exp
  return 1.0 / (1 + np.exp(-x))


Predicciones:  [8, 4, 5, 7, 7, 0, 6, 2, 7, 4]
Y_Real      :  [8 4 8 7 7 0 6 2 7 4]
Precisión: 0.9030714285714285


In [43]:
recall = calculate_recall(y_pred, y_prueba, average='macro')
accuracy = calculate_accuracy(y_pred, y_prueba)
precision = calculate_precision(y_pred, y_prueba, average='macro')
f1 = calculate_f1_score(y_pred, y_prueba, average='macro')
training_time = end_time - start_time

pd.DataFrame({
    'Metric': ['Recall', 'Accuracy', 'Precision', 'F1-score', 'Training time'],
    'Score': [recall, accuracy, precision, f1, training_time]
})

,Metric,Score
0,Recall,0.902060
1,Accuracy,0.903071
2,Precision,0.901813
3,F1-score,0.901741
4,Training time,1481.431912


# Comparacion con TensorFlow

### Hiperparametros utilizados en la biblioteca creada anteriormente:

capas=[330,15,12,32,10], funciones_capas=['tanh', 'relu', 'sigmoid', 'sigmoid'], epochs=500

In [75]:
import time
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.utils import to_categorical

# Preprocesar los datos
X_train = X_entrenamiento.reshape((X_entrenamiento.shape[0], -1))
X_test = X_prueba.reshape((X_prueba.shape[0], -1))

# Codificar las etiquetas de entrenamiento y prueba
y_train_encoded = to_categorical(y_entrenamiento)
y_test_encoded = to_categorical(y_prueba)

# Crear el modelo MLP
model = Sequential()
model.add(Flatten(input_shape=(330,)))
model.add(Dense(15, activation='tanh'))
model.add(Dense(12, activation='relu'))
model.add(Dense(32, activation='sigmoid'))
model.add(Dense(10, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', 'Recall', 'Precision'])

# Entrenar el modelo
start = time.time()
model.fit(X_train, y_train_encoded, epochs=500, validation_data=(X_test, y_test_encoded), verbose=2)
end = time.time()


Epoch 1/500
1750/1750 - 1s - loss: 0.7270 - accuracy: 0.8140 - recall: 0.6691 - precision: 0.9283 - val_loss: 0.3408 - val_accuracy: 0.9072 - val_recall: 0.8908 - val_precision: 0.9282 - 1s/epoch - 628us/step
Epoch 2/500
1750/1750 - 1s - loss: 0.2917 - accuracy: 0.9177 - recall: 0.9053 - precision: 0.9347 - val_loss: 0.2875 - val_accuracy: 0.9203 - val_recall: 0.9096 - val_precision: 0.9353 - 840ms/epoch - 480us/step
Epoch 3/500
1750/1750 - 1s - loss: 0.2476 - accuracy: 0.9293 - recall: 0.9191 - precision: 0.9431 - val_loss: 0.2659 - val_accuracy: 0.9243 - val_recall: 0.9139 - val_precision: 0.9397 - 817ms/epoch - 467us/step
Epoch 4/500
1750/1750 - 1s - loss: 0.2261 - accuracy: 0.9344 - recall: 0.9255 - precision: 0.9475 - val_loss: 0.2612 - val_accuracy: 0.9248 - val_recall: 0.9157 - val_precision: 0.9369 - 813ms/epoch - 464us/step
Epoch 5/500
1750/1750 - 1s - loss: 0.2121 - accuracy: 0.9387 - recall: 0.9298 - precision: 0.9505 - val_loss: 0.2598 - val_accuracy: 0.9264 - val_recall: 0

In [78]:
# Evalúa el modelo en los datos de prueba
loss, accuracy, recall, precision = model.evaluate(X_test, y_test_encoded, verbose=0)

pd.DataFrame({
    'Metric': ['Recall', 'Accuracy', 'Precision', 'Training time'],
    'Score': [recall, accuracy, precision, end-start]
})

,Metric,Score
0,Recall,0.910143
1,Accuracy,0.913143
2,Precision,0.919734
3,Training time,397.516268


## Conclusiones al comparar los resultados obtenidos del MLP sin bibliotecas contra el MLP de TensorFlow

En términos de rendimiento, el modelo MLP implementado manualmente obtuvo un buen desempeño, con una precisión, recall y exactitud alrededor del 90%. Sin embargo, los resultados mejoraron significativamente cuando se utilizó TensorFlow para implementar el MLP, obteniendo una precisión, recall y exactitud superiores al 90%, alcanzando valores cercanos al 91% y 91.3% respectivamente.

Esto sugiere que el uso de TensorFlow, una biblioteca de aprendizaje automático altamente optimizada, puede proporcionar mejoras sustanciales en el rendimiento del modelo MLP en comparación con la implementación manual. La precisión mejorada y el recall superior indican una capacidad mejorada para clasificar correctamente las imágenes en el conjunto de datos MNIST.

Además, el tiempo de entrenamiento del modelo MLP implementado con TensorFlow fue considerablemente menor en comparación con la implementación manual. Mientras que el modelo manual requirió aproximadamente 1481.43 segundos para entrenarse, el modelo TensorFlow se entrenó en tan solo 397.52 segundos. Esta diferencia en el tiempo de entrenamiento demuestra la eficiencia de TensorFlow y cómo puede acelerar el proceso de entrenamiento de modelos de aprendizaje automático.